# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 



**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Подрядок выполнения работы:
1) Загрузка файла, знакомстов с данными
2) Обработка текста: лемматизирование и очистка от знаков препинания и лишних пробелов. Создание тренировочной и тестовой выборок
3) Обучение моделей LinearRegression и DecisionTreeClassifier. Анализ результатов работы моделе на основании метрики f1 (не ниже 0.75)
4) Вывод

## Подготовка

In [1]:

import warnings
import pandas as pd
import numpy as np
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
#следующие записи нужны для корректной работы progress_apply()
nltk.download('averaged_perceptron_tagger')
from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#импорт файлов
pd.set_option('display.max_columns', 50) 
pth1_1 = '/datasets/toxic_comments.csv'
pth2_1 = 'toxic_comments.csv'

if os.path.exists(pth1_1):
    tweet= pd.read_csv(pth1_1, index_col=[0], parse_dates=[0], sep=',', decimal=';')
elif os.path.exists(pth2_1):
    tweet = pd.read_csv(pth2_1, index_col=[0], parse_dates=[0], sep=',', decimal=';')
else:
    print('Something is wrong')

In [3]:
display(tweet.head(5))
tweet.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
tweet['toxic'].value_counts()
display(tweet['toxic'].value_counts(), round((sum(tweet['toxic']) / len(tweet) * 100),2))

0    143106
1     16186
Name: toxic, dtype: int64

10.16

Создадим функции по лемметизированию текста и его очистки от знаков препинания и лишних пробелов

In [5]:
stopwords = set(nltk_stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  

def clear_text(text):
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    tokens = word_tokenize(pattern)  
    pos_tags = nltk.pos_tag(tokens)  
    for word, pos in pos_tags:
        lemm.append(lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)))  
    return " ".join(lemm)

<b>Комментарий студента: удалено</b></font>

In [7]:
tweet['corpus_lemm'] = tweet['text'].progress_apply(clear_text)

100%|██████████| 159292/159292 [10:15<00:00, 258.81it/s]


In [8]:
display(tweet)

,text,toxic,corpus_lemm
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,And for the second time of ask when your view ...
159447,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...
159449,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...


Разобьём датасет на тестовую и тренировочную выборки


In [9]:
X = tweet['corpus_lemm']
y = tweet['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print('Train data shape:', X_train.shape, y_train.shape)
print('Test data shape:', X_test.shape, y_test.shape)

Train data shape: (127433,) (127433,)
Test data shape: (31859,) (31859,)


In [11]:
display(X_train, y_train)

45208     You claim to have scavenge the UN and NGO webs...
60971     Please do not vandalize page a you do with thi...
92333           large moon Shouldn t it say large know moon
74833     Isn t bake cook According to the article apple...
7210      I be sure the judge smile too When you conside...
                                ...                        
119984    First of all editor TheRedPenOfDoom be not a M...
103791                             and decide to become gay
132068    John Bull v Adolf Hitler Look at the picture o...
147023    October You have a lot to learn about the way ...
122063    I m sorry Tip the deduction be obvious and wro...
Name: corpus_lemm, Length: 127433, dtype: object

45208     0
60971     0
92333     0
74833     0
7210      0
         ..
119984    0
103791    1
132068    0
147023    0
122063    0
Name: toxic, Length: 127433, dtype: int64

Проведём векторизацию

In [12]:
vec_tf_idf = TfidfVectorizer(ngram_range=(1,1), stop_words=list(stopwords),
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True,
               smooth_idf=True, sublinear_tf=True )

In [13]:
train_vec = vec_tf_idf.fit_transform(X_train)
test_vec = vec_tf_idf.transform(X_test)
 

In [14]:
print(f"Размер тренировочного датасета: {train_vec.shape}")
print(f"Размер тестового датасета: {test_vec.shape}")

Размер тренировочного датасета: (127433, 39329)
Размер тестового датасета: (31859, 39329)


## Обучение

LogisticRegression

In [15]:
parameters = {'C': np.linspace(10, 20, num = 11, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
grid = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
grid.fit(train_vec, y_train)


Fitting 5 folds for each of 11 candidates, totalling 55 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]),
                         'max_iter': [1000]},
             scoring='f1', verbose=2)

In [16]:
print(f"Наилучший показатель f1 на кросс-валидации : {grid.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {grid.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.775
Параметр регуляризации для лучшей модели: {'C': 13.0, 'max_iter': 1000}


DecisionTreeClassifier

In [17]:
param_grid = [
    {   
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8]
    }
]
dtc = DecisionTreeClassifier()
grid1 = GridSearchCV(dtc, param_grid,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
grid1.fit(train_vec, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [2, 4, 6, 8]}],
             scoring='f1', verbose=2)

In [18]:
print(f"Наилучший показатель f1 на кросс-валидации : {grid1.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {grid1.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.565
Параметр регуляризации для лучшей модели: {'criterion': 'gini', 'max_depth': 8}


Лучшая модель по метрике f1 - модель логистической регрессии.  
Протестируем её на тестовой выборке

In [19]:
lrm = LogisticRegression(C=13, max_iter=1000)
lrm.fit(train_vec, y_train)
predict = lrm.predict(test_vec)
f1_lr = f1_score(y_test, predict)
print(f"Показатель f1 на тестовой выборке: {f1_lr:.3f}")

Показатель f1 на тестовой выборке: 0.777


## Выводы

В данной работы были осуществлены следующие действия:
1) Загрузка данных  
В таблице 2 столбца с текстами и оценкой их токсичности.   
Всего в таблице 159292 записи. Среди всех текстов 10.16% токсичные
2) Обработка данных и создание выборок.    
Была проведена лемматизация текстов и их очистка от знаков препинания и лишних пробелов.  
Массив данных был разделён на тренировочные (80% данных) и тестовые (20% данных) выборки
3) Обучение моделей, и оценка их работы.   
Были обучены модели Linear Regression и Decision Tree Classifier. С помощью метрики f1 видим оценили работу моделей:  
    - Linear Regression с параметрами 'C': 13.0, 'max_iter': 1000; f1 = 0.775  
    - Decision Tree Classifier c параметрами 'criterion': 'gini', 'max_depth': 8; f1 = 0.56  
Видим, что метрике f1 лучше проявила себя модель линейной регрессии.  
Применили её к тестовой выборке и получили значение метрики f1 = 0.777  
4) Было продемонстрировано, что среди рассмотренных моделей лучше всего оценивает токсичность комментариев 
модель линейной регрессии с парамтрами 'C': 13.0, 'max_iter': 1000